In [ ]:
import pandas as pd
import requests
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier



ModuleNotFoundError: No module named 'import_ipynb'

In [ ]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)


In [ ]:



API_KEY = "8edf5a3557214b83a8d24412250109"
local = "Curitiba"  # pode ser "Curitiba, Brazil" ou latitude
url = "http://api.weatherapi.com/v1/forecast.json"
params = {
    "key": API_KEY,
    "q": local,
    "lang": "pt",     # opcional: traduz descrições para português
    "aqi": "no"       # opcional: inclui dados de qualidade do ar
}

try:
    response = requests.get(url, params=params)
    response.raise_for_status()
    dados = response.json()
    
    # Exemplo de como acessar alguns dados:
    # temperatura = dados["current"]["temp_c"]
    # descricao = dados["current"]["condition"]["text"]
    # umidade = dados["current"]["humidity"]
    # vento = dados["current"]["wind_kph"]

    # print(f"Tempo em {dados['location']['name']}: {descricao.capitalize()}")
    # print(f"Temperatura: {temperatura} °C")
    # print(f"Umidade: {umidade}%")
    # print(f"Vento: {vento} km/h")
   # print(dados)

except requests.exceptions.HTTPError as err:
    print(f"Erro HTTP: {err} — código: {response.status_code}")
except ValueError:
    print("Erro ao decodificar JSON.")
except Exception as e:
    print(f"Ocorreu um erro: {e}")

df = pd.json_normalize(dados)
df

    


,location.name,location.region,location.country,location.lat,location.lon,location.tz_id,location.localtime_epoch,location.localtime,current.last_updated_epoch,current.last_updated,current.temp_c,current.temp_f,current.is_day,current.condition.text,current.condition.icon,current.condition.code,current.wind_mph,current.wind_kph,current.wind_degree,current.wind_dir,current.pressure_mb,current.pressure_in,current.precip_mm,current.precip_in,current.humidity,current.cloud,current.feelslike_c,current.feelslike_f,current.windchill_c,current.windchill_f,current.heatindex_c,current.heatindex_f,current.dewpoint_c,current.dewpoint_f,current.vis_km,current.vis_miles,current.uv,current.gust_mph,current.gust_kph,forecast.forecastday
0,Curitiba,Parana,Brazil,-25.4167,-49.25,America/Sao_Paulo,1759030087,2025-09-28 00:28,1759029300,2025-09-28 00:15,13.0,55.4,0,Encoberto,//cdn.weatherapi.com/weather/64x64/night/122.png,1009,5.8,9.4,79,E,1023.0,30.21,0.0,0.0,94,100,12.3,54.2,9.5,49.1,10.6,51.1,9.1,48.4,10.0,6.0,0.0,8.2,13.1,"[{'date': '2025-09-28', 'date_epoch': 17590176..."


In [ ]:
forecast_list = dados["forecast"]["forecastday"]
df_forecast = pd.json_normalize(forecast_list)
df_forecast


,date,date_epoch,hour,day.maxtemp_c,day.maxtemp_f,day.mintemp_c,day.mintemp_f,day.avgtemp_c,day.avgtemp_f,day.maxwind_mph,day.maxwind_kph,day.totalprecip_mm,day.totalprecip_in,day.totalsnow_cm,day.avgvis_km,day.avgvis_miles,day.avghumidity,day.daily_will_it_rain,day.daily_chance_of_rain,day.daily_will_it_snow,day.daily_chance_of_snow,day.condition.text,day.condition.icon,day.condition.code,day.uv,astro.sunrise,astro.sunset,astro.moonrise,astro.moonset,astro.moon_phase,astro.moon_illumination,astro.is_moon_up,astro.is_sun_up
0,2025-09-28,1759017600,"[{'time_epoch': 1759028400, 'time': '2025-09-2...",23.3,74.0,10.6,51.0,16.2,61.1,6.5,10.4,0.0,0.0,0.0,10.0,6.0,72,0,0,0,0,Parcialmente nublado,//cdn.weatherapi.com/weather/64x64/day/116.png,1003,1.8,06:00 AM,06:15 PM,10:01 AM,No moonset,Waxing Crescent,31,0,0


In [ ]:
# Extrair a lista de horas do primeiro dia de previsão
hour_list = df_forecast['hour'][0]  # Pega a lista de horas do primeiro dia

# Agora normalize essa lista para criar o dataframe de horas
df_hour = pd.json_normalize(hour_list)

# Converter para datetime
df_hour["time"] = pd.to_datetime(df_hour["time"])

# Definir como índice
df_hour = df_hour.set_index("time")

# Selecionar apenas colunas numéricas
df_numeric = df_hour.select_dtypes(include="number")

# Reamostrar de 5h em 5h e calcular média
df_5h = df_numeric.resample("5h").mean().reset_index()

print(df_5h.head())
df_5h

                 time    time_epoch  temp_c  temp_f  is_day  wind_mph  \
0 2025-09-28 00:00:00  1.759036e+09   11.12   51.98     0.0      5.68   
1 2025-09-28 05:00:00  1.759054e+09   12.96   55.30     0.6      4.34   
2 2025-09-28 10:00:00  1.759072e+09   21.90   71.42     1.0      5.40   
3 2025-09-28 15:00:00  1.759090e+09   19.48   67.04     0.8      3.08   
4 2025-09-28 20:00:00  1.759106e+09   15.80   60.50     0.0      1.55   

   wind_kph  wind_degree  pressure_mb  pressure_in  precip_mm  precip_in  \
0     9.140         77.6       1021.0       30.148        0.0        0.0   
1     6.980         62.8       1020.6       30.142        0.0        0.0   
2     8.620        312.6       1017.4       30.046        0.0        0.0   
3     4.960        169.6       1016.0       30.000        0.0        0.0   
4     2.525         63.0       1018.0       30.065        0.0        0.0   

   snow_cm  humidity  cloud  feelslike_c  feelslike_f  windchill_c  \
0      0.0     91.40  79.40       

,time,time_epoch,temp_c,temp_f,is_day,wind_mph,wind_kph,wind_degree,pressure_mb,pressure_in,precip_mm,precip_in,snow_cm,humidity,cloud,feelslike_c,feelslike_f,windchill_c,windchill_f,heatindex_c,heatindex_f,dewpoint_c,dewpoint_f,will_it_rain,chance_of_rain,will_it_snow,chance_of_snow,vis_km,vis_miles,gust_mph,gust_kph,uv,condition.code
0,2025-09-28 00:00:00,1.759036e+09,11.12,51.98,0.0,5.68,9.140,77.6,1021.0,30.148,0.0,0.0,0.0,91.40,79.40,9.56,49.16,9.56,49.16,10.64,51.12,9.18,48.540,0.0,0.0,0.0,0.0,10.0,6.0,7.940,12.780,0.00,1006.6
1,2025-09-28 05:00:00,1.759054e+09,12.96,55.30,0.6,4.34,6.980,62.8,1020.6,30.142,0.0,0.0,0.0,81.00,55.00,12.54,54.58,12.54,54.58,12.96,55.30,9.66,49.380,0.0,0.0,0.0,0.0,10.0,6.0,5.480,8.820,0.74,1004.2
2,2025-09-28 10:00:00,1.759072e+09,21.90,71.42,1.0,5.40,8.620,312.6,1017.4,30.046,0.0,0.0,0.0,47.60,36.00,22.84,73.10,21.92,71.42,23.48,74.26,10.24,50.400,0.0,0.0,0.0,0.0,10.0,6.0,6.260,10.060,6.58,1001.8
3,2025-09-28 15:00:00,1.759090e+09,19.48,67.04,0.8,3.08,4.960,169.6,1016.0,30.000,0.0,0.0,0.0,64.00,15.20,20.36,68.64,19.48,67.04,20.36,68.64,12.22,54.000,0.0,0.0,0.0,0.0,10.0,6.0,5.440,8.740,1.12,1000.0
4,2025-09-28 20:00:00,1.759106e+09,15.80,60.50,0.0,1.55,2.525,63.0,1018.0,30.065,0.0,0.0,0.0,77.25,22.75,15.85,60.50,15.85,60.50,15.85,60.50,11.90,53.375,0.0,0.0,0.0,0.0,10.0,6.0,3.275,5.275,0.00,1001.5


In [ ]:
colunas_importantes = [
    "time",
    "feelslike_c",
    "humidity",
    "cloud",
    "uv",
    "wind_kph",
    "will_it_rain",
    "chance_of_rain",
]

df_limpo = df_5h[[c for c in colunas_importantes if c in df_5h.columns]]
df_limpo.sample(5)

,time,feelslike_c,humidity,cloud,uv,wind_kph,will_it_rain,chance_of_rain
2,2025-09-28 10:00:00,22.84,47.60,36.00,6.58,8.620,0.0,0.0
0,2025-09-28 00:00:00,9.56,91.40,79.40,0.00,9.140,0.0,0.0
3,2025-09-28 15:00:00,20.36,64.00,15.20,1.12,4.960,0.0,0.0
4,2025-09-28 20:00:00,15.85,77.25,22.75,0.00,2.525,0.0,0.0
1,2025-09-28 05:00:00,12.54,81.00,55.00,0.74,6.980,0.0,0.0


In [ ]:
df_tabela1 = pd.DataFrame({
    "estilos": ["casual", "esportivo", "social", "streetwear"],
    "data e hora": df_limpo["time"].values[:4],
    "temperatura": df_limpo["feelslike_c"].values[:4],
    "chance de chuva": df_limpo["chance_of_rain"].values[:4],
    "umidade": df_limpo["humidity"].values[:4],
    "uv": df_limpo["uv"].values[:4],
})

df_tabela1

,estilos,data e hora,temperatura,chance de chuva,umidade,uv
0,casual,2025-09-28 00:00:00,9.56,0.0,91.4,0.00
1,esportivo,2025-09-28 05:00:00,12.54,0.0,81.0,0.74
2,social,2025-09-28 10:00:00,22.84,0.0,47.6,6.58
3,streetwear,2025-09-28 15:00:00,20.36,0.0,64.0,1.12


In [ ]:
# Garante que 'data e hora' seja do tipo datetime
df_tabela1['data e hora'] = pd.to_datetime(df_tabela1['data e hora'])

# Extrai hora, minuto, dia, etc.
df_tabela1['hora'] = df_tabela1['data e hora'].dt.hour
df_tabela1['dia'] = df_tabela1['data e hora'].dt.day
df_tabela1['mes'] = df_tabela1['data e hora'].dt.month

In [ ]:
print(df_tabela1.columns.tolist())

['estilos', 'data e hora', 'temperatura', 'chance de chuva', 'umidade', 'uv', 'hora', 'dia', 'mes']


In [ ]:
#transformando variáveis e em um número categorico 

X = df_tabela1[['temperatura', 'chance de chuva', 'umidade', 'uv', 'hora']]  # todas numéricas
y = df_tabela1['estilos']  # variável alvo

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

clf = DecisionTreeClassifier(max_depth=4, random_state=42)
clf.fit(X_train, y_train)


,criterion,'gini'
,splitter,'best'
,max_depth,4
,min_samples_split,2
,min_samples_leaf,1
,min_weight_fraction_leaf,0.0
,max_features,None
,random_state,42
,max_leaf_nodes,None
,min_impurity_decrease,0.0
,class_weight,None


In [ ]:
# Supondo que df_tabela1 já tenha a coluna 'hora' extraída
X_dummies = df_tabela1[['temperatura', 'chance de chuva', 'umidade', 'uv', 'hora']]

In [ ]:
df_tabela1['predicao_estilo'] = clf.predict(X_dummies)
print(df_tabela1)


      estilos         data e hora  temperatura  chance de chuva  umidade  \
0      casual 2025-09-28 00:00:00         9.56              0.0     91.4   
1   esportivo 2025-09-28 05:00:00        12.54              0.0     81.0   
2      social 2025-09-28 10:00:00        22.84              0.0     47.6   
3  streetwear 2025-09-28 15:00:00        20.36              0.0     64.0   

     uv  hora  dia  mes predicao_estilo  
0  0.00     0   28    9          casual  
1  0.74     5   28    9      streetwear  
2  6.58    10   28    9          social  
3  1.12    15   28    9      streetwear  
